In [ ]:
# Intall and import libraries
from sklearn.model_selection import train_test_split
from tensorflow.python.client import device_lib
from keras.preprocessing.text import Tokenizer
from tensorflow.keras import layers
from operator import itemgetter
from tensorflow import keras
from ast import literal_eval
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import json
import re

pd.set_option('display.max_columns', None)
directory='/content' ### Set Directory

In [ ]:
%%bash
# Script to make sure data files can be used

directory='/content' ### Must be the same as the variable directory

if [ ! -d $directory ]; then
mkdir -p $directory
fi

cd $directory
wget https://storage.googleapis.com/yantk-siads-699/699_final.tar.gz
tar -xvzf ${directory}/699_final.tar.gz .
mkdir ${directory}/699
mkdir ${directory}/699/699
mv ${directory}/Data\ 20230612 ${directory}/699/699
mv ${directory}/model ${directory}/699/699

In [ ]:
# Import data from selected countries including Canada, UK and USA
# Data from https://www.kaggle.com/datasets/rsrishav/youtube-trending-video-dataset?select=US_youtube_trending_data.csv

country_list = ['CA','GB','US']

def get_categories(country):
    r = open(directory + '/699/699/Data 20230612/Youtube/' + country + '_category_id.json', "r")
    data = json.loads(r.read())
    categories_list = []
    for category in data['items']:
        categories_list.append([category['id'], category['snippet']['title']])
    categories = pd.DataFrame(categories_list).rename(columns={0: country+"_category_id", 1: country+"_category_name"})
    return categories

for country in country_list:
    exec(f'{country}_categories = get_categories(country)')              #country_categories = individual country's category dataframe
    exec(f'{country}_youtube_df = pd.read_csv(directory + "/699/699/Data 20230612/Youtube/" + country + "_youtube_trending_data.csv")') #country_youtube_df = individual country's YouTube dataframe

In [ ]:
# Combine all countries data, drop the duplicates and remove columns
df = pd.concat([CA_youtube_df,GB_youtube_df,US_youtube_df])

In [ ]:
# Retain only one entry per video, or one entry per video per day

df = df.drop_duplicates(subset=['video_id'])
df = df.drop(['thumbnail_link','comments_disabled','ratings_disabled'], axis=1)

In [ ]:
# Map the categoryId to categoryName with US's category table
categories = US_categories.rename(columns={"US_category_id": "categoryId", "US_category_name": "categoryName"})

categories['categoryId'] = categories['categoryId'].astype(str).astype(int)
df = df.merge(categories, on='categoryId', how='left')

In [ ]:
# Split the tags
def split_tags(tags):
  tag_list = tags.lower().split('|')
  return tag_list

df['original_tag_list']=df['tags'].apply(split_tags)

In [ ]:
# Detect each tag's language with regex and only retain those with English characters
def en_tags(tag_list):
  en_tag_list = []
  for tag in tag_list:
    en_tag = re.findall(r'(^[0-9a-zA-Z .?\-","]+$)',tag)
    en_tag_list += en_tag
  return en_tag_list

df['en_tag_list']=df['original_tag_list'].apply(en_tags)

In [ ]:
# Drop rows without tags (optional)
df = df[df['tags']!="[None]"]
df = df[df['en_tag_list'].str.len()!=0]

In [ ]:
df_keras_multi_model = df[['video_id','title','description','en_tag_list']]

In [ ]:
# Use only the most common 85000 tags in the dataset for training, in order to increase efficient

tokenizer = Tokenizer(split='|', filters='')

texts = df['tags']
tokenizer.fit_on_texts(texts)

tag_words = json.loads(tokenizer.get_config()['word_counts'])
result = dict(sorted(tag_words.items(), key=itemgetter(1), reverse=True))

#Amend the line below to get the % of tags
result_split = dict(sorted(tag_words.items(), key=itemgetter(1), reverse=True)[:85000])

#20000 tags = 60%
#50000 tags = 70%
#85000 tags = 80%

print('Total number of tags (with duplicates):',sum(result.values()))
print('Top x number of unique tags used:',len(result_split))
print('Accounts of x%:',sum(result_split.values())/sum(result.values()))

top_tags_lst = result_split.keys()

Total number of tags (with duplicates): 1048288
Top x number of unique tags used: 85000
Accounts of x%: 0.791721359015843


In [ ]:
# Remove tags which are not most most common based on the previous step

def keep_most_used(en_tag_lst):
    return [item for item in en_tag_lst if item in top_tags_lst]

df_keras_multi_model['top_tags'] = df_keras_multi_model['en_tag_list'].apply(keep_most_used)

<ipython-input-18-6af1e6d5f73f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_keras_multi_model['top_tags'] = df_keras_multi_model['en_tag_list'].apply(keep_most_used)


In [ ]:
# Truncate descriptions
def get_first_x_words(sentence):
    x=130
    return ' '.join(str(sentence).split(' ')[:x])

df_keras_multi_model['description'] = df_keras_multi_model['description'].apply(get_first_x_words)

<ipython-input-19-6d0c2e21aeff>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_keras_multi_model['description'] = df_keras_multi_model['description'].apply(get_first_x_words)


In [ ]:
# Split the data set into 80% for training, 10% for validation, and 10% for testing

train_df, test_df = train_test_split(df_keras_multi_model, test_size=0.2, random_state=42)
validation_df = test_df.sample(frac=0.5, random_state=42)
test_df = test_df.drop(validation_df.index)

In [ ]:
# Output dataframe
test_df.to_csv(directory + '/699/699/Data 20230612/Youtube/test_set_youtube.csv')
train_df.to_csv(directory + '/699/699/Data 20230612/Youtube/train_set_youtube.csv')
validation_df.to_csv(directory + '/699/699/Data 20230612/Youtube/validation_set_youtube.csv')